In [2]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

In [3]:
import xgboost 
xgboost.__version__

'0.90'

In [4]:
data = pd.read_csv('../Field_training_data_justMMETSP.csv')
labels = pd.read_csv('../Field_training_labels_justMMETSP.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams.csv')

In [5]:
features = features['Pfam'].str.split('.').str[0]

In [6]:
len(features)

1046

In [7]:
le = LabelEncoder()

In [8]:
train_data = train_data[features]

In [9]:
from sklearn.model_selection import train_test_split
from sklearn import svm

In [10]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [11]:
#from https://scikit-learn.org/stable/modules/cross_validation.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [12]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import make_scorer, f1_score
from scipy import stats
from xgboost import XGBClassifier
import numpy as np

# Assuming X and y are your features and labels

np.random.seed(7)

#model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
clf = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)

# Define a custom scoring function for F1 score
f1_scorer = make_scorer(f1_score, average='weighted')

# Create a StratifiedShuffleSplit cross-validator
kf = StratifiedShuffleSplit(n_splits=6, random_state=7)

# Perform cross-validation and store the F1 scores
f1_scores = cross_val_score(clf, X, y, cv=kf, scoring=f1_scorer)

# save F1 scores, mean, and standard error to csv file
data = {'F1_Scores': f1_scores}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('f1_scores_results_oldModel.csv', index=False)

/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre

In [13]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.metrics import make_scorer, f1_score
from xgboost import XGBClassifier
import numpy as np
import pandas as pd

# Assuming X and y are your features and labels

np.random.seed(7)

#model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
clf = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)

# Define a custom scoring function for F1 score
def f1_scorer(y_true, y_pred):
    return f1_score(y_true, y_pred, average=None)

# Create a StratifiedShuffleSplit cross-validator
kf = StratifiedShuffleSplit(n_splits=6, random_state=7)

# Initialize an empty list to store F1 scores for each class
all_f1_scores = []

# Perform cross-validation and store the F1 scores for each fold
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    fold_f1_scores = f1_scorer(y_test, y_pred)
    all_f1_scores.append(fold_f1_scores)

# Convert the list of arrays to a numpy array
f1_scores = np.array(all_f1_scores)

# Save the F1 scores for each class separately
class_names = [f'Class_{i}' for i in range(f1_scores.shape[1])]
data = {class_name: f1_scores[:, i] for i, class_name in enumerate(class_names)}

df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('f1_scores_results_oldModel_byClass.csv', index=False)

/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/utils/__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the pre